In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import os
import xlwings as xw

In [2]:
minerals = ["Manganese", "Lithium", "Cobalt", "Nickel", "Lanthanides"]
data_directory = "./data" # Set this to your own directory

In [3]:
combined = None

mining_path = os.path.join(data_directory, "SPGlobal_MetalsAndMiningProperties-Combined-Dec-2024.xlsx")
for i, mineral in enumerate(minerals):
    data = pd.read_excel(mining_path, sheet_name = mineral)
    print(f"Read {mineral}")

    # data.dropna(subset="Primary Reserves and Resources", inplace = True)
    data["Reserves & Resources As Of Date"] = pd.to_datetime(data["Reserves & Resources As Of Date"])
    data["Primary Mineral"] = mineral
    data["Year"] = data["Reserves & Resources As Of Date"].dt.year


    if i == 0:
        combined = data
        continue

    combined = pd.concat([combined, data], ignore_index=True)

combined = combined.sort_values(by = "Year").query("`Activity Status` == 'Active'")
combined["Property ID"] = combined["Property ID"].astype(int)

Read Manganese
Read Lithium
Read Cobalt
Read Nickel
Read Lanthanides


In [4]:
locations = pd.read_excel("data/MiningPropertyLocations.xlsx")
coords = locations[["KeyMineProject","Latitude (degrees)", "Longitude (degrees)"]]

combined = combined.merge(coords, left_on = "Property ID", right_on="KeyMineProject" ,how = "inner")
combined.drop("KeyMineProject", axis=1, inplace=True, errors="ignore")
combined

,Property,Property ID,Owner(s),Country/Region,Country/Region Risk Score & Outlook,Development Stage,Activity Status,Commodity(s),Primary Reserves and Resources,Unit,Total In-Situ Value ($M),Reserves & Resources As Of Date,Primary Mineral,Year,Latitude (degrees),Longitude (degrees)
0,Lithium Two,79958,"New Age Metals Inc., Mineral Resources Ltd.",Canada,1.4 Moderate - No Change,Target Outline,Active,"Lithium, Caesium, Rubidium, Tantalum",7600,tonnes,123.180800,1947-12-31,Lithium,1947,50.61363,-95.46443
1,Falcon West,90157,Grid Metals Corp.,Canada,1.4 Moderate - No Change,Target Outline,Active,"Lithium, Caesium, Tantalum, Rubidium",4000,tonnes,64.832000,1955-12-31,Lithium,1955,49.56729,-95.68944
2,Irgon,79892,QMC Quantum Minerals Corp,Canada,1.4 Moderate - No Change,Target Outline,Active,"Lithium, Tantalum",16420,tonnes,266.135360,1956-12-31,Lithium,1956,50.60720,-95.39696
3,Chrome-Puddy,89842,"Green Bridge Metals Corp., Pavey Ark Minerals...",Canada,1.4 Moderate - No Change,Exploration,Active,"Nickel, Chromium, Copper, Cobalt, Platinum, Ir...",73482,tonnes,1384.357942,1966-12-31,Nickel,1966,49.96907,-89.52303
4,Chrome-Puddy,89842,"Green Bridge Metals Corp., Pavey Ark Minerals...",Canada,1.4 Moderate - No Change,Exploration,Active,"Nickel, Chromium, Copper, Cobalt, Platinum, Ir...",4627,tonnes,1384.357942,1966-12-31,Cobalt,1966,49.96907,-89.52303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,Brothers,90684,Critica Ltd.,Australia,1.4 Moderate - No Change,Reserves Development,Active,"Lanthanides, Dysprosium, Terbium, Thorium, U3O...",2900000,tonnes,357712.977515,2025-02-11,Lanthanides,2025,-28.43831,117.30562
657,Caldeira,90102,Meteoric Resources NL,Brazil,2.3 Elevated - No Change,Prefeas/Scoping,Active,"Lanthanides, Neodymium, Praseodymium, Terbium,...",2673400,tonnes,133670.000000,2025-03-12,Lanthanides,2025,-21.98932,-46.49532
658,Colossus,90981,Viridis Mining & Minerals Ltd.,Brazil,2.3 Elevated - No Change,Prefeas/Scoping,Active,"Lanthanides, Yttrium, U3O8, Thorium, Praseodym...",1236700,tonnes,61835.000000,2025-01-22,Lanthanides,2025,-21.86427,-46.54833
659,Browns Range,37953,Northern Minerals Ltd.,Australia,1.4 Moderate - No Change,Preproduction,Active,"Lanthanides, Dysprosium, Yttrium, Terbium, Lut...",90458,tonnes,4590.922240,2025-01-15,Lanthanides,2025,-18.96977,128.94522


In [22]:
C = combined.copy()
usca = C[C["Primary Mineral"].isin(["Cobalt", "Nickel", "Manganese"]) & 
         C["Country/Region"].isin(["USA", "Canada"])]
C[10:30]

,Property,Property ID,Owner(s),Country/Region,Country/Region Risk Score & Outlook,Development Stage,Activity Status,Commodity(s),Primary Reserves and Resources,Unit,Total In-Situ Value ($M),Reserves & Resources As Of Date,Primary Mineral,Year,Latitude (degrees),Longitude (degrees)
10,Bouvier,59623,Mining Equities Pty Ltd,Canada,1.4 Moderate - No Change,Target Outline,Active,Lithium,11431,tonnes,185.273648,1979-12-31,Lithium,1979,48.45182,-78.09764
11,Crean Hill 3,91456,"SPC Nickel Corp, Vale S.A.",Canada,1.4 Moderate - No Change,Target Outline,Active,"Nickel, Copper",72150,tonnes,1618.180659,1989-12-31,Nickel,1989,46.43776,-81.31067
12,Gochager Lake,80201,"Fathom Nickel Inc., BioELife Corp.",Canada,1.4 Moderate - No Change,Target Outline,Active,"Cobalt, Copper, Nickel, Platinum, Palladium, Gold",13010,tonnes,214.542410,1990-12-31,Nickel,1990,55.76232,-104.94067
13,Pomalaa East,28852,PT Vale Indonesia Tbk,Indonesia,2.1 Elevated - No Change,Construction Started,Active,"Nickel, Cobalt",2690000,tonnes,44359.652699,1993-12-31,Nickel,1993,-4.22492,121.68711
14,Miriam,29806,"Future Battery Minerals Ltd., Corazon Mining ...",Australia,1.4 Moderate - No Change,Target Outline,Active,"Lithium, Nickel, Spodumene, Gold, Caesium, Tan...",3900,tonnes,64.313251,1996-06-21,Nickel,1996,-31.05194,121.10314
15,Punta Gorda,28013,Cubaniquel,Cuba,3.0 High - No Change,Operating,Active,"Nickel, Cobalt",4093000,tonnes,80215.987760,1999-12-31,Nickel,1999,20.62977,-74.86241
16,Punta Gorda,28013,Cubaniquel,Cuba,3.0 High - No Change,Operating,Active,"Nickel, Cobalt",341000,tonnes,80215.987760,1999-12-31,Cobalt,1999,20.62977,-74.86241
17,Nonoc,27118,Philnico Mining & Indl Corp,Philippines,2.2 Elevated - No Change,Feasibility,Active,"Nickel, Cobalt, Iron Ore",1584000,tonnes,32014.824235,2000-02-23,Nickel,2000,9.83278,125.61056
18,Nonoc,27118,Philnico Mining & Indl Corp,Philippines,2.2 Elevated - No Change,Feasibility,Active,"Nickel, Cobalt, Iron Ore",158000,tonnes,32014.824235,2000-02-23,Cobalt,2000,9.83278,125.61056
19,Loveland,84460,Intl Explorers & Prospectors I,Canada,1.4 Moderate - No Change,Target Outline,Active,"Nickel, Cobalt, Copper, Palladium",5262,tonnes,124.852742,2002-03-31,Nickel,2002,48.66136,-81.62469


In [6]:
C = combined.copy()
usca = C[C["Primary Mineral"].isin(["Cobalt", "Nickel", "Manganese"]) & 
         C["Country/Region"].isin(["USA", "Canada"])]

In [27]:
import networkx as nx
from geopy.distance import geodesic
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder

def build(df, required_metals = {"Cobalt", "Nickel", "Manganese"} ,max_distance_per_cluster=7000):  # km
    
    simplified_df = df.rename(columns={'Latitude (degrees)': 'Lat', 
                       'Longitude (degrees)': 'Lon',
                        'Primary Reserves and Resources': 'Reserves',
                        'Property ID': 'ID',
                        'Property': 'Name'})

    
    simplified_df['Normalized_Reserves'] = simplified_df['Reserves'] / simplified_df['Reserves'].max()

    coords = simplified_df[['Lat', 'Lon']]
    clustering = DBSCAN(eps=max_distance_per_cluster/1000).fit(coords)
    simplified_df['Cluster'] = clustering.labels_

    fig = go.Figure()



    def check_coverage(sites):
        found = set()
        for s in sites:
            for metal in required_metals:
                if metal.lower() in s['Commodity(s)'].lower():
                    found.add(metal)
        return required_metals.issubset(found)

    for cluster in simplified_df['Cluster'].unique():
        if cluster == -1:
            continue
        
        cluster_data = simplified_df[simplified_df['Cluster'] == cluster].copy()
        if cluster_data.empty:
            continue

        fig.add_trace(go.Scattergeo(
            lon=cluster_data['Lon'],
            lat=cluster_data['Lat'],
            customdata=cluster_data[['Reserves', 'Name', 'Commodity(s)', "Primary Mineral"]],
            mode='markers+text',
            hovertemplate="<b>%{customdata[1]}</b><br>" +
                          "Latitude: %{lat}<br>" +
                          "Longitude: %{lon}<br>" +
                          "Commodities: %{customdata[2]}<br>" +
                          "Primary Commodity: %{customdata[3]}<br>" +
                          "Reserves: %{customdata[0]:,.0f} tonnes",
            textposition="bottom center",
            marker=dict(size=8, symbol='circle'),
            name=f'Cluster {cluster}'
        ))

        # Greedy selection algorithm with metal constraint
        mines = cluster_data.to_dict(orient='records')
        start = max(mines, key=lambda x: x['Reserves'])

        visited = [start]
        unvisited = [m for m in mines if m['ID'] != start['ID']]
        total_distance = 0

        while unvisited:
            best_mine = None
            best_score = float('-inf')
            best_distance = None

            for candidate in unvisited:
                dist = geodesic(
                    (visited[-1]['Lat'], visited[-1]['Lon']),
                    (candidate['Lat'], candidate['Lon'])
                ).km
                if dist == 0:
                    continue
                score = candidate['Reserves'] / dist
                if score > best_score:
                    best_score = score
                    best_mine = candidate
                    best_distance = dist

            if best_distance and total_distance + best_distance <= max_distance_per_cluster:
                total_distance += best_distance
                visited.append(best_mine)
                unvisited = [u for u in unvisited if u['ID'] != best_mine['ID']]
            else:
                break

            # If constraint satisfied, we can optionally stop early
            if check_coverage(visited):
                break

        # Try adding backup mines just to fulfill constraints
        if not check_coverage(visited):
            for backup in sorted(unvisited, key=lambda x: -x['Reserves']):
                dist = geodesic(
                    (visited[-1]['Lat'], visited[-1]['Lon']),
                    (backup['Lat'], backup['Lon'])
                ).km
                if total_distance + dist > max_distance_per_cluster:
                    continue
                visited.append(backup)
                total_distance += dist
                if check_coverage(visited):
                    break

        # Draw path
        for i in range(len(visited) - 1):
            node1 = visited[i]
            node2 = visited[i + 1]
            fig.add_trace(go.Scattergeo(
                lon=[node1['Lon'], node2['Lon']],
                lat=[node1['Lat'], node2['Lat']],
                mode='lines',
                line=dict(width=2, color='gray'),
                showlegend=False
            ))

    fig.update_layout(
        title="Constraint-Aware Resource Route: Covering Key Metals",
        margin=dict(l=5, r=5, t=30, b=30),
        geo=dict(
            scope='world',
            showland=True,
            landcolor='lightgray',
            countrycolor='white',
            lonaxis=dict(range=[simplified_df["Lon"].min() - 10, simplified_df["Lon"].max() + 10]),
            lataxis=dict(range=[simplified_df["Lat"].min() - 10, simplified_df["Lat"].max() + 10])
        ),
        template="plotly_white"
    )

    fig.show()


In [28]:
build(C, required_metals={"Cobalt", "Nickel" ,"Manganese"})